In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/")

In [2]:
import pandas as pd
import numpy as np
import joblib
from scipy import stats

from exp_vaep.domain.vaep.vaep import *

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Load VAEP Data

In [3]:
exp_vaep_data = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/schema_chains_exp_vaep_values.csv")
exp_vaep_data.head()

,match_id,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,scores,concedes,offensive_value,defensive_value,vaep_value
0,202101_BrisbaneLions_Sydney,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,0.0,0.246684,0.158121,0.000000,-0.000000,0.000000
1,202101_BrisbaneLions_Sydney,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,0.0,0.000000,0.362654,-0.246684,-0.204533,-0.451217
2,202101_BrisbaneLions_Sydney,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,0.0,0.668677,0.081827,0.306023,-0.081827,0.224196
3,202101_BrisbaneLions_Sydney,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,0.0,0.456306,0.099139,-0.212371,-0.017312,-0.229683
4,202101_BrisbaneLions_Sydney,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,0.0,1.176206,0.069984,0.719900,0.029155,0.749055


In [4]:
exp_vaep_data['Season'] = exp_vaep_data['match_id'].apply(lambda x: int(x[:4]))
exp_vaep_2021 = exp_vaep_data[exp_vaep_data['Season'] == 2021]
exp_vaep_2022 = exp_vaep_data[exp_vaep_data['Season'] == 2022]
exp_vaep_2023 = exp_vaep_data[exp_vaep_data['Season'] == 2023]

In [5]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/player_stats_202319.csv")

In [6]:
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats_2021 = player_stats[player_stats['Season'] == 2021]
player_stats_2022 = player_stats[player_stats['Season'] == 2022]
player_stats_2023 = player_stats[player_stats['Season'] == 2023]

Individual Actions

In [7]:
exp_vaep_data['vaep_value'].max(), exp_vaep_data['vaep_value'].min()

(5.7585473, -5.3814387)

In [8]:
exp_vaep_data.sort_values('vaep_value', ascending = False).head()

,match_id,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,scores,concedes,offensive_value,defensive_value,vaep_value,Season
18546,202102_GoldCoast_NorthMelbourne,1,113.0,113.0,Gold Coast,Jack Lukosius,75.0,0.0,68.0,0.0,Contested Mark,effective,0.0,5.864003,0.105455,5.864003,-0.105455,5.758547,2021
238792,202117_WestCoast_NorthMelbourne,4,1715.0,6946.0,North Melbourne,Todd Goldstein,77.0,0.0,70.0,0.0,Contested Mark,effective,0.0,5.771208,0.058066,5.771208,-0.058066,5.713142,2021
496598,202211_Collingwood_Carlton,1,1051.0,1051.0,Collingwood,Steele Sidebottom,73.0,-5.0,68.0,-2.0,Contested Mark,effective,0.0,5.921538,0.210826,5.921538,-0.210826,5.710712,2022
882001,202315_Fremantle_Essendon,4,1152.0,6613.0,Essendon,Zach Merrett,52.0,1.0,44.0,0.0,Free For,effective,0.0,3.259767,0.198297,3.010187,2.684864,5.695051,2023
724174,202303_Hawthorn_NorthMelbourne,2,34.0,1684.0,North Melbourne,Nick Larkey,83.0,-5.0,80.0,-10.0,Contested Mark,effective,0.0,5.766105,0.104063,5.766105,-0.104063,5.662043,2023


In [9]:
exp_vaep_data.sort_values('vaep_value', ascending = True).head()

,match_id,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,scores,concedes,offensive_value,defensive_value,vaep_value,Season
564388,202216_GoldCoast_Collingwood,2,521.0,2481.0,Collingwood,Brody Mihocek,73.0,7.0,67.0,-14.0,Kick,clanger,0.0,0.000000,1.138345,-4.531992,-0.849447,-5.381439,2022
702930,202302_Adelaide_Richmond,2,738.0,2607.0,Richmond,Kamdyn McIntosh,79.0,-1.0,-47.0,38.0,Kick,ineffective,0.0,0.160270,0.826992,-4.726501,-0.481293,-5.207793,2023
128713,202109_PortAdelaide_WesternBulldogs,3,816.0,4792.0,Port Adelaide,Todd Marshall,76.0,-15.0,-46.0,1.0,Kick,ineffective,0.0,0.294168,0.974771,-3.932121,-0.723129,-4.655250,2021
383255,202203_GreaterWesternSydney_GoldCoast,4,952.0,6485.0,Greater Western Sydney,Harry Himmelberg,-62.0,3.0,60.0,-8.0,Spoil,effective,0.0,0.451018,0.388357,-3.926690,-0.297670,-4.224361,2022
86652,202106_Melbourne_Richmond,1,821.0,821.0,Melbourne,Tom McDonald,68.0,19.0,68.0,26.0,Kick,clanger,0.0,0.577678,0.719419,-3.539314,-0.670399,-4.209713,2021


Match Level

In [10]:
def get_match_player_level_vaep(vaep_data, match_id, shots = True):
    
    match = get_match(vaep_data, match_id)
    if shots == False:
        match = match[match['action_type'] != "Shot"]
    match_vaep = get_player_vaep(match)
    match_vaep_ranking = get_match_rankings(match_vaep)
    
    return match_vaep_ranking

In [66]:
match_id = "2022F4_Geelong_Sydney"

In [12]:
match_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id)
match_vaep_ranking.sort_values('vaep_value', ascending = False).head()

,player,team,vaep_value,offensive_value,defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_vaep_value,normalised_offensive_value,normalised_defensive_value,z_vaep_value,z_offensive_value,z_defensive_value
5,Chad Warner,Sydney,13.569964,13.891235,-0.321271,69,1.0,1.0,35.0,1.000000,1.000000,0.302138,2.448048,2.555069,-0.506856
24,Mark Blicavs,Geelong,12.392599,11.351990,1.040609,49,2.0,4.0,10.0,0.920200,0.830159,0.492781,2.127088,1.889899,0.385276
13,Isaac Smith,Geelong,12.125486,13.375887,-1.250401,69,3.0,2.0,42.0,0.902095,0.965530,0.172073,2.054271,2.420071,-1.115504
30,Patrick Dangerfield,Geelong,11.997718,12.976029,-0.978310,58,4.0,3.0,40.0,0.893435,0.938785,0.210162,2.019440,2.315326,-0.937264
4,Cameron Guthrie,Geelong,10.065827,9.639613,0.426214,36,5.0,6.0,22.0,0.762494,0.715624,0.406775,1.492790,1.441332,-0.017198


In [13]:
match_noshot_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id, shots=False)
match_noshot_vaep_ranking.sort_values('vaep_value', ascending = False).head()

,player,team,vaep_value,offensive_value,defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_vaep_value,normalised_offensive_value,normalised_defensive_value,z_vaep_value,z_offensive_value,z_defensive_value
30,Patrick Dangerfield,Geelong,12.211371,13.136062,-0.924692,57,1.0,1.0,41.0,1.000000,1.000000,0.217668,2.581818,3.032604,-0.966736
24,Mark Blicavs,Geelong,10.190843,9.289252,0.901591,47,2.0,3.0,15.0,0.852292,0.734190,0.473321,1.958590,1.840992,0.276916
13,Isaac Smith,Geelong,10.162234,10.842248,-0.680014,65,3.0,2.0,37.0,0.850201,0.841500,0.251919,1.949766,2.322058,-0.800117
5,Chad Warner,Sydney,8.778989,9.003736,-0.224747,66,4.0,4.0,34.0,0.749081,0.714461,0.315650,1.523106,1.752549,-0.490092
36,Tom Atkins,Geelong,8.436854,6.878736,1.558118,35,5.0,7.0,8.0,0.724069,0.567626,0.565225,1.417575,1.094296,0.723994


In [14]:
match_stats = player_stats[player_stats['Match_ID'] == match_id]
match_stats[['Team', 'Player', "Goals", "Behinds", "Disposals"]]

,Team,Player,Goals,Behinds,Disposals
18967,Geelong,Jack Henry,0,0,10
18968,Sydney,Hayden McLean,1,0,4
18969,Sydney,Dylan Stephens,0,0,12
18970,Geelong,Zach Guthrie,0,1,15
18971,Geelong,Gryan Miers,0,1,18
18972,Sydney,Nick Blakey,0,0,15
18973,Sydney,James Rowbottom,0,0,16
18974,Geelong,Mark O'Connor,0,0,13
18975,Sydney,Tom McCartin,0,0,9
18976,Geelong,Sam De Koning,1,0,16


Player Ratings

In [15]:
players_vaep = get_player_vaep(exp_vaep_data)
players_vaep.sort_values('vaep_value', ascending = False).head(5)

,player,team,vaep_value,offensive_value,defensive_value,num_actions
145,Christian Petracca,Melbourne,709.998664,730.169195,-20.170531,4045
568,Marcus Bontempelli,Western Bulldogs,698.406899,683.820860,14.586039,3813
769,Shai Bolton,Richmond,691.864155,708.788685,-16.924529,2585
309,Jack Macrae,Western Bulldogs,592.712200,536.761904,55.950297,4389
831,Tom Liberatore,Western Bulldogs,581.846562,510.057805,71.788755,3447


In [16]:
players_noshot_vaep = get_player_vaep(exp_vaep_data[exp_vaep_data['action_type'] != "Shot"])
players_noshot_vaep.sort_values('vaep_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions
568,Marcus Bontempelli,Western Bulldogs,579.563406,559.980984,19.582422,3668
309,Jack Macrae,Western Bulldogs,555.599993,498.162852,57.437140,4355
145,Christian Petracca,Melbourne,538.370159,561.308892,-22.938734,3848
769,Shai Bolton,Richmond,515.966580,531.444184,-15.477604,2388
831,Tom Liberatore,Western Bulldogs,507.320864,433.746129,73.574734,3379
147,Clayton Oliver,Melbourne,504.011041,476.754233,27.256810,3994
500,Lachie Neale,Brisbane Lions,496.337404,473.755435,22.581972,3540
554,Luke Parker,Sydney,474.994739,449.312317,25.682423,3471
719,Rory Laird,Adelaide,448.260670,407.447094,40.813576,3852
890,Zach Merrett,Essendon,442.558741,402.813016,39.745728,3768


Normalise for minutes played

In [17]:
player_minutes = get_player_minutes(player_stats)
players_vaep_mins = players_vaep.merge(player_minutes, how = 'left', on = "player")
player_ratings = convert_vaep_to_rating(players_vaep_mins)
player_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
769,Shai Bolton,Richmond,691.864155,708.788685,-16.924529,2585,5086,60,13.603306,13.936073,-0.332767
568,Marcus Bontempelli,Western Bulldogs,698.406899,683.820860,14.586039,3813,5575,66,12.527478,12.265845,0.261633
683,Patrick Dangerfield,Geelong,403.448420,407.637369,-4.188949,2199,3272,47,12.330331,12.458355,-0.128024
145,Christian Petracca,Melbourne,709.998664,730.169195,-20.170531,4045,5853,67,12.130509,12.475127,-0.344619
55,Ben Cunnington,North Melbourne,211.787102,193.894950,17.892150,1240,1830,24,11.573066,10.595352,0.977713
138,Charlie Curnow,Carlton,462.572301,497.661437,-35.089134,1236,4015,44,11.521103,12.395054,-0.873951
831,Tom Liberatore,Western Bulldogs,581.846562,510.057805,71.788755,3447,5094,64,11.422194,10.012913,1.409281
147,Clayton Oliver,Melbourne,560.823429,532.716488,28.106943,4051,5052,58,11.101018,10.544665,0.556353
194,Darcy Parish,Essendon,465.772934,463.586720,2.186213,3416,4212,52,11.058237,11.006332,0.051904
309,Jack Macrae,Western Bulldogs,592.712200,536.761904,55.950297,4389,5421,67,10.933632,9.901529,1.032103


In [19]:
player_minutes = get_player_minutes(player_stats)
players_noshot_vaep_mins = players_noshot_vaep.merge(player_minutes, how = 'left', on = "player")
player_noshot_ratings = convert_vaep_to_rating(players_noshot_vaep_mins)
player_noshot_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
683,Patrick Dangerfield,Geelong,353.803921,357.684401,-3.880479,2133,3272,47,10.813078,10.931675,-0.118597
568,Marcus Bontempelli,Western Bulldogs,579.563406,559.980984,19.582422,3668,5575,66,10.395756,10.044502,0.351254
309,Jack Macrae,Western Bulldogs,555.599993,498.162852,57.437140,4355,5421,67,10.249031,9.189501,1.059530
55,Ben Cunnington,North Melbourne,187.544218,170.057429,17.486788,1216,1830,24,10.248318,9.292756,0.955562
769,Shai Bolton,Richmond,515.966580,531.444184,-15.477604,2388,5086,60,10.144840,10.449158,-0.304318
194,Darcy Parish,Essendon,426.795792,425.887278,0.908514,3376,4212,52,10.132854,10.111284,0.021570
147,Clayton Oliver,Melbourne,504.011041,476.754233,27.256810,3994,5052,58,9.976466,9.436940,0.539525
831,Tom Liberatore,Western Bulldogs,507.320864,433.746129,73.574734,3379,5094,64,9.959185,8.514844,1.444341
423,Joel Selwood,Geelong,318.463048,293.714593,24.748454,2160,3233,44,9.850388,9.084893,0.765495
719,Rory Laird,Adelaide,448.260670,407.447094,40.813576,3852,4557,58,9.836749,8.941126,0.895624


In [20]:
player_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
769,Shai Bolton,Richmond,691.864155,708.788685,-16.924529,2585,5086,60,13.603306,13.936073,-0.332767
145,Christian Petracca,Melbourne,709.998664,730.169195,-20.170531,4045,5853,67,12.130509,12.475127,-0.344619
683,Patrick Dangerfield,Geelong,403.448420,407.637369,-4.188949,2199,3272,47,12.330331,12.458355,-0.128024
138,Charlie Curnow,Carlton,462.572301,497.661437,-35.089134,1236,4015,44,11.521103,12.395054,-0.873951
568,Marcus Bontempelli,Western Bulldogs,698.406899,683.820860,14.586039,3813,5575,66,12.527478,12.265845,0.261633
829,Tom J. Lynch,Richmond,391.134030,431.025518,-39.891488,1048,3635,41,10.760221,11.857648,-1.097427
810,Toby Greene,Greater Western Sydney,465.968207,516.014375,-50.046167,1665,4383,48,10.631262,11.773086,-1.141824
361,Jake Stringer,Essendon,423.459464,455.331674,-31.872209,1541,3957,50,10.701528,11.506992,-0.805464
795,Taylor Walker,Adelaide,439.497572,491.780742,-52.283169,1421,4318,51,10.178267,11.389086,-1.210819
194,Darcy Parish,Essendon,465.772934,463.586720,2.186213,3416,4212,52,11.058237,11.006332,0.051904


In [21]:
player_noshot_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
683,Patrick Dangerfield,Geelong,353.803921,357.684401,-3.880479,2133,3272,47,10.813078,10.931675,-0.118597
769,Shai Bolton,Richmond,515.966580,531.444184,-15.477604,2388,5086,60,10.144840,10.449158,-0.304318
194,Darcy Parish,Essendon,426.795792,425.887278,0.908514,3376,4212,52,10.132854,10.111284,0.021570
568,Marcus Bontempelli,Western Bulldogs,579.563406,559.980984,19.582422,3668,5575,66,10.395756,10.044502,0.351254
138,Charlie Curnow,Carlton,369.647021,392.352091,-22.705068,1010,4015,44,9.206651,9.772157,-0.565506
145,Christian Petracca,Melbourne,538.370159,561.308892,-22.938734,3848,5853,67,9.198192,9.590106,-0.391914
829,Tom J. Lynch,Richmond,317.946161,347.657707,-29.711545,858,3635,41,8.746799,9.564174,-0.817374
147,Clayton Oliver,Melbourne,504.011041,476.754233,27.256810,3994,5052,58,9.976466,9.436940,0.539525
55,Ben Cunnington,North Melbourne,187.544218,170.057429,17.486788,1216,1830,24,10.248318,9.292756,0.955562
500,Lachie Neale,Brisbane Lions,496.337404,473.755435,22.581972,3540,5104,59,9.724479,9.282042,0.442437


In [22]:
player_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
757,Sam Taylor,Greater Western Sydney,368.239021,89.026011,279.213009,2277,4816,51,7.646159,1.848547,5.797612
23,Aliir Aliir,Port Adelaide,407.437394,145.192319,262.245077,2478,5871,61,6.939830,2.473042,4.466787
816,Tom Barrass,West Coast,263.589691,68.148105,195.441586,2084,4501,49,5.856247,1.514066,4.342181
819,Tom Clurey,Port Adelaide,123.153912,29.717951,93.435961,810,2210,25,5.572575,1.344704,4.227872
735,Sam Collins,Gold Coast,341.001066,101.336373,239.664694,2237,5697,60,5.985625,1.778767,4.206858
257,Harris Andrews,Brisbane Lions,406.789869,145.811149,260.978720,2825,6217,64,6.543186,2.345362,4.197824
193,Darcy Moore,Collingwood,405.043244,194.466019,210.577224,2495,5125,54,7.903283,3.794459,4.108824
133,Charlie Ballard,Gold Coast,313.507855,117.669392,195.838464,2167,5222,57,6.003597,2.253340,3.750258
367,James Blanck,Hawthorn,74.681010,18.044777,56.636233,428,1535,19,4.865212,1.175556,3.689657
652,Nick Murray,Adelaide,206.539117,75.451188,131.087929,1208,3694,46,5.591205,2.042534,3.548672


In [23]:
player_noshot_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
757,Sam Taylor,Greater Western Sydney,368.239021,89.026011,279.213009,2277,4816,51,7.646159,1.848547,5.797612
23,Aliir Aliir,Port Adelaide,405.557720,143.622341,261.935382,2474,5871,61,6.907813,2.446301,4.461512
816,Tom Barrass,West Coast,262.513190,66.881514,195.631675,2083,4501,49,5.832330,1.485926,4.346405
819,Tom Clurey,Port Adelaide,123.153912,29.717951,93.435961,810,2210,25,5.572575,1.344704,4.227872
735,Sam Collins,Gold Coast,341.001066,101.336373,239.664694,2237,5697,60,5.985625,1.778767,4.206858
257,Harris Andrews,Brisbane Lions,405.317484,144.323251,260.994233,2822,6217,64,6.519503,2.321429,4.198074
193,Darcy Moore,Collingwood,403.630581,191.749232,211.881348,2486,5125,54,7.875719,3.741448,4.134270
133,Charlie Ballard,Gold Coast,311.571752,115.648709,195.923044,2165,5222,57,5.966521,2.214644,3.751878
367,James Blanck,Hawthorn,74.681010,18.044777,56.636233,428,1535,19,4.865212,1.175556,3.689657
652,Nick Murray,Adelaide,203.645776,72.511819,131.133957,1204,3694,46,5.512880,1.962962,3.549918


VAEP by Action

In [25]:
action_vaep = get_vaep_action_summary(exp_vaep_data)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-1900.076972,-1044.922249,-855.154728,2975,0.003139,-0.638681,-0.351234,-0.287447,-0.014796,-0.009082,-0.063975
5,Handball,-10135.821909,-5277.444374,-4858.377529,166573,0.175731,-0.060849,-0.031682,-0.029167,-0.078928,-0.045870,-0.363462
12,Tackle,-12.639873,2.538690,-15.178563,293,0.000309,-0.043139,0.008664,-0.051804,-0.000098,0.000022,-0.001136
7,Kick,1723.364590,9353.602703,-7630.238121,220438,0.232557,0.007818,0.042432,-0.034614,0.013420,0.081300,-0.570829
0,Carry,12806.711508,14918.499467,-2111.787959,149555,0.157777,0.085632,0.099753,-0.014120,0.099727,0.129668,-0.157986
4,Gather,5354.506771,4586.297237,768.209530,37682,0.039754,0.142097,0.121711,0.020387,0.041696,0.039863,0.057471
11,Spoil,5892.335872,907.880443,4984.455438,37977,0.040065,0.155155,0.023906,0.131249,0.045884,0.007891,0.372894
13,Uncontested Mark,20608.951816,19441.174255,1167.777561,105273,0.111061,0.195767,0.184674,0.011093,0.160483,0.168979,0.087363
6,Hard Ball Get,9828.955040,6163.404872,3665.550162,48623,0.051296,0.202146,0.126759,0.075387,0.076539,0.053571,0.274225
9,Loose Ball Get,26828.965034,18099.955668,8729.009371,100997,0.106550,0.265641,0.179213,0.086428,0.208919,0.157321,0.653030


In [26]:
match_data = get_match(exp_vaep_data, match_id)
match_action_vaep = get_vaep_action_summary(match_data)
match_action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-0.636182,-0.848427,0.212245,3,0.001858,-0.212061,-0.282809,0.070748,-0.003013,-0.004458,0.010197
5,Handball,-19.480666,-13.108350,-6.372316,280,0.173375,-0.069574,-0.046816,-0.022758,-0.092267,-0.068875,-0.306163
4,Gather,1.708319,3.819077,-2.110758,59,0.036533,0.028955,0.064730,-0.035776,0.008091,0.020066,-0.101413
7,Kick,14.273632,25.819972,-11.546341,374,0.231579,0.038165,0.069037,-0.030873,0.067604,0.135665,-0.554753
11,Spoil,5.984199,2.431596,3.552602,58,0.035913,0.103176,0.041924,0.061252,0.028343,0.012776,0.170687
0,Carry,28.899800,32.604539,-3.704740,265,0.164087,0.109056,0.123036,-0.013980,0.136878,0.171313,-0.177997
6,Hard Ball Get,11.586375,8.478221,3.108154,80,0.049536,0.144830,0.105978,0.038852,0.054877,0.044547,0.149334
12,Uncontested Mark,32.074055,30.420526,1.653529,193,0.119505,0.166187,0.157619,0.008568,0.151913,0.159838,0.079445
9,Loose Ball Get,44.137275,25.841757,18.295518,190,0.117647,0.232301,0.136009,0.096292,0.209048,0.135780,0.879022
1,Contested Mark,7.779834,5.564547,2.215286,16,0.009907,0.486240,0.347784,0.138455,0.036848,0.029238,0.106435


In [27]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-60.263744,-26.313116,-33.950629,880,0.217553,-0.068482,-0.029901,-0.038580,-0.084879,-0.036037,1.683180
2,Error,-14.260604,-8.628101,-5.632502,25,0.006180,-0.570424,-0.345124,-0.225300,-0.020085,-0.011817,0.279244
8,Knock On,0.743619,0.194399,0.549220,12,0.002967,0.061968,0.016200,0.045768,0.001047,0.000266,-0.027229
11,Spoil,5.941700,5.409779,0.531921,17,0.004203,0.349512,0.318222,0.031289,0.008369,0.007409,-0.026371
1,Contested Mark,11.668496,11.203646,0.464849,17,0.004203,0.686382,0.659038,0.027344,0.016435,0.015344,-0.023046
3,Free For,55.069205,47.279695,7.789509,57,0.014091,0.966126,0.829468,0.136658,0.077562,0.064752,-0.386183
0,Carry,59.752981,69.821532,-10.068550,622,0.153770,0.096066,0.112253,-0.016187,0.084159,0.095624,0.499171
6,Hard Ball Get,68.736032,61.880276,6.855756,298,0.073671,0.230658,0.207652,0.023006,0.096811,0.084748,-0.339890
4,Gather,79.722872,67.321212,12.401660,309,0.076391,0.258003,0.217868,0.040135,0.112286,0.092199,-0.614841
7,Kick,87.122083,109.034789,-21.912706,838,0.207169,0.103964,0.130113,-0.026149,0.122707,0.149328,1.086372


In [28]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-41.085667,-23.758644,-17.327023,554,0.213982,-0.074162,-0.042886,-0.031276,-0.165655,-0.097918,-3.220329
1,Error,-5.095362,-2.844225,-2.251136,7,0.002704,-0.727909,-0.406318,-0.321591,-0.020544,-0.011722,-0.418387
7,Knock On,-1.048524,-1.766504,0.717980,6,0.002317,-0.174754,-0.294417,0.119663,-0.004228,-0.007280,0.133441
10,Spoil,0.005665,1.365814,-1.360150,8,0.003090,0.000708,0.170727,-0.170019,0.000023,0.005629,-0.252792
6,Kick,6.860328,19.315889,-12.455561,589,0.227501,0.011647,0.032794,-0.021147,0.027661,0.079608,-2.314939
3,Gather,12.045017,8.854340,3.190677,93,0.035921,0.129516,0.095208,0.034308,0.048565,0.036492,0.593006
5,Hard Ball Get,14.884251,14.107661,0.776590,77,0.029741,0.193302,0.183216,0.010086,0.060013,0.058143,0.144334
11,Uncontested Mark,22.210457,20.976320,1.234137,183,0.070684,0.121369,0.114625,0.006744,0.089551,0.086451,0.229372
2,Free For,36.478582,23.003196,13.475386,50,0.019312,0.729572,0.460064,0.269508,0.147080,0.094804,2.504480
9,Shot,44.276781,44.899965,-0.623183,51,0.019699,0.868172,0.880391,-0.012219,0.178522,0.185049,-0.115822


In [29]:
player = "Darcy Moore"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
7,Kick,-29.259462,0.404354,-29.663816,586,0.234870,-0.049931,0.000690,-0.050621,-0.072238,0.002079,-0.140869
2,Error,-4.450402,-0.698558,-3.751845,6,0.002405,-0.741734,-0.116426,-0.625307,-0.010987,-0.003592,-0.017817
10,Shot,1.412663,2.716787,-1.304124,9,0.003607,0.156963,0.301865,-0.144903,0.003488,0.013970,-0.006193
4,Gather,6.261387,6.363537,-0.102150,44,0.017635,0.142304,0.144626,-0.002322,0.015459,0.032723,-0.000485
5,Handball,8.141813,9.524432,-1.382619,275,0.110220,0.029607,0.034634,-0.005028,0.020101,0.048977,-0.006566
8,Knock On,15.089293,5.375135,9.714158,59,0.023647,0.255751,0.091104,0.164647,0.037254,0.027640,0.046131
0,Carry,22.610675,33.220567,-10.609892,292,0.117034,0.077434,0.113769,-0.036335,0.055823,0.170830,-0.050385
6,Hard Ball Get,24.329606,4.614945,19.714661,98,0.039279,0.248261,0.047091,0.201170,0.060067,0.023731,0.093622
9,Loose Ball Get,42.121950,11.106061,31.015890,192,0.076954,0.219385,0.057844,0.161541,0.103994,0.057111,0.147290
12,Uncontested Mark,50.638813,41.380809,9.258004,346,0.138677,0.146355,0.119598,0.026757,0.125021,0.212792,0.043965


In [30]:
player = "Taylor Walker"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value").round(2)

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-52.79,-39.91,-12.88,199,0.14,-0.27,-0.20,-0.06,-0.12,-0.08,0.25
11,Spoil,-11.42,-7.98,-3.43,33,0.02,-0.35,-0.24,-0.10,-0.03,-0.02,0.07
4,Gather,-6.26,-4.20,-2.06,40,0.03,-0.16,-0.10,-0.05,-0.01,-0.01,0.04
2,Error,-4.10,-2.51,-1.58,7,0.00,-0.59,-0.36,-0.23,-0.01,-0.01,0.03
8,Knock On,7.20,7.49,-0.30,18,0.01,0.40,0.42,-0.02,0.02,0.02,0.01
0,Carry,7.99,11.73,-3.73,104,0.07,0.08,0.11,-0.04,0.02,0.02,0.07
7,Kick,16.54,26.47,-9.92,215,0.15,0.08,0.12,-0.05,0.04,0.05,0.19
6,Hard Ball Get,31.39,37.37,-5.98,108,0.08,0.29,0.35,-0.06,0.07,0.08,0.11
9,Loose Ball Get,43.61,46.26,-2.65,116,0.08,0.38,0.40,-0.02,0.10,0.09,0.05
3,Free For,83.37,76.53,6.85,56,0.04,1.49,1.37,0.12,0.19,0.16,-0.13


By Season

In [31]:
players_2021 = get_player_vaep(exp_vaep_2021)
players_2022 = get_player_vaep(exp_vaep_2022)
players_2023 = get_player_vaep(exp_vaep_2023)

In [32]:
player_minutes_2021 = get_player_minutes(player_stats_2021)
player_minutes_2022 = get_player_minutes(player_stats_2022)
player_minutes_2023 = get_player_minutes(player_stats_2023)

In [33]:
players_2021 = players_2021.merge(player_minutes_2021, how = "left", on = "player")
players_2022 = players_2022.merge(player_minutes_2022, how = "left", on = "player")
players_2023 = players_2023.merge(player_minutes_2023, how = "left", on = "player")

In [34]:
player_ratings_2021 = convert_vaep_to_rating(players_2021)
player_ratings_2022 = convert_vaep_to_rating(players_2022)
player_ratings_2023 = convert_vaep_to_rating(players_2023)

2021

In [35]:
player_ratings_2021.sort_values('vaep_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
112,Clayton Oliver,Melbourne,260.623654,248.981274,11.642383,1702,2211,25,11.787592,11.261025,0.526566
110,Christian Petracca,Melbourne,256.786801,272.486627,-15.699826,1524,2237,25,11.479070,12.180895,-0.701825
232,Jack Macrae,Western Bulldogs,255.927568,238.855886,17.071681,1872,2154,26,11.881503,11.088945,0.792557
420,Marcus Bontempelli,Western Bulldogs,249.544111,245.625472,3.918640,1569,2193,26,11.379120,11.200432,0.178689
154,Dayne Zorko,Brisbane Lions,245.674467,235.034118,10.640349,1199,1900,23,12.930235,12.370217,0.560018
618,Tom Liberatore,Western Bulldogs,233.560643,210.332832,23.227810,1284,1967,25,11.873952,10.693077,1.180875
598,Toby Greene,Greater Western Sydney,218.587898,232.174140,-13.586242,663,1627,18,13.435028,14.270076,-0.835049
245,Jack Steele,St Kilda,217.559197,203.308461,14.250736,1379,1870,22,11.634182,10.872110,0.762071
492,Ollie Wines,Port Adelaide,209.356850,210.176423,-0.819573,1648,1855,24,11.286084,11.330265,-0.044182
567,Shai Bolton,Richmond,208.880452,215.502932,-6.622480,847,1628,20,12.830495,13.237281,-0.406786


In [36]:
player_ratings_2021.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
598,Toby Greene,Greater Western Sydney,218.587898,232.174140,-13.586242,663,1627,18,13.435028,14.270076,-0.835049
154,Dayne Zorko,Brisbane Lions,245.674467,235.034118,10.640349,1199,1900,23,12.930235,12.370217,0.560018
567,Shai Bolton,Richmond,208.880452,215.502932,-6.622480,847,1628,20,12.830495,13.237281,-0.406786
268,Jake Stringer,Essendon,194.206106,205.465425,-11.259320,661,1526,19,12.726481,13.464314,-0.737832
503,Patrick Dangerfield,Geelong,137.166198,136.572310,0.593889,749,1080,15,12.700574,12.645584,0.054990
42,Ben Cunnington,North Melbourne,154.370564,143.311014,11.059549,880,1237,15,12.479431,11.585369,0.894062
232,Jack Macrae,Western Bulldogs,255.927568,238.855886,17.071681,1872,2154,26,11.881503,11.088945,0.792557
618,Tom Liberatore,Western Bulldogs,233.560643,210.332832,23.227810,1284,1967,25,11.873952,10.693077,1.180875
462,Nat Fyfe,Fremantle,144.003956,139.383867,4.620090,790,1220,15,11.803603,11.424907,0.378696
112,Clayton Oliver,Melbourne,260.623654,248.981274,11.642383,1702,2211,25,11.787592,11.261025,0.526566


In [37]:
player_ratings_2021.sort_values('offensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
110,Christian Petracca,Melbourne,256.786801,272.486627,-15.699826,1524,2237,25,11.479070,12.180895,-0.701825
112,Clayton Oliver,Melbourne,260.623654,248.981274,11.642383,1702,2211,25,11.787592,11.261025,0.526566
420,Marcus Bontempelli,Western Bulldogs,249.544111,245.625472,3.918640,1569,2193,26,11.379120,11.200432,0.178689
232,Jack Macrae,Western Bulldogs,255.927568,238.855886,17.071681,1872,2154,26,11.881503,11.088945,0.792557
154,Dayne Zorko,Brisbane Lions,245.674467,235.034118,10.640349,1199,1900,23,12.930235,12.370217,0.560018
598,Toby Greene,Greater Western Sydney,218.587898,232.174140,-13.586242,663,1627,18,13.435028,14.270076,-0.835049
567,Shai Bolton,Richmond,208.880452,215.502932,-6.622480,847,1628,20,12.830495,13.237281,-0.406786
2,Aaron Naughton,Western Bulldogs,200.725128,215.274540,-14.549412,607,2041,25,9.834646,10.547503,-0.712857
618,Tom Liberatore,Western Bulldogs,233.560643,210.332832,23.227810,1284,1967,25,11.873952,10.693077,1.180875
492,Ollie Wines,Port Adelaide,209.356850,210.176423,-0.819573,1648,1855,24,11.286084,11.330265,-0.044182


In [38]:
player_ratings_2021.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
598,Toby Greene,Greater Western Sydney,218.587898,232.174140,-13.586242,663,1627,18,13.435028,14.270076,-0.835049
268,Jake Stringer,Essendon,194.206106,205.465425,-11.259320,661,1526,19,12.726481,13.464314,-0.737832
567,Shai Bolton,Richmond,208.880452,215.502932,-6.622480,847,1628,20,12.830495,13.237281,-0.406786
503,Patrick Dangerfield,Geelong,137.166198,136.572310,0.593889,749,1080,15,12.700574,12.645584,0.054990
589,Taylor Walker,Adelaide,152.401258,169.531589,-17.130330,441,1369,16,11.132305,12.383608,-1.251302
154,Dayne Zorko,Brisbane Lions,245.674467,235.034118,10.640349,1199,1900,23,12.930235,12.370217,0.560018
110,Christian Petracca,Melbourne,256.786801,272.486627,-15.699826,1524,2237,25,11.479070,12.180895,-0.701825
42,Ben Cunnington,North Melbourne,154.370564,143.311014,11.059549,880,1237,15,12.479431,11.585369,0.894062
462,Nat Fyfe,Fremantle,144.003956,139.383867,4.620090,790,1220,15,11.803603,11.424907,0.378696
305,Jeremy Cameron,Geelong,131.296360,140.939805,-9.643444,386,1237,15,10.614095,11.393679,-0.779583


In [39]:
player_ratings_2021.sort_values('defensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
194,Harris Andrews,Brisbane Lions,199.096075,71.459961,127.636113,1109,2224,23,8.952162,3.213128,5.739034
18,Aliir Aliir,Port Adelaide,181.933166,76.006224,105.926944,1042,2315,24,7.858884,3.283206,4.575678
263,Jake Lever,Melbourne,183.445366,82.954609,100.490758,1141,2472,25,7.420929,3.355769,4.065160
557,Sam Taylor,Greater Western Sydney,122.807288,24.724011,98.083278,829,1781,19,6.895412,1.388209,5.507203
617,Tom Jonas,Port Adelaide,131.264200,43.761497,87.502701,985,2297,24,5.714593,1.905159,3.809434
540,Sam Collins,Gold Coast,112.503566,25.839008,86.664558,725,1788,20,6.292146,1.445135,4.847011
13,Alex Keath,Western Bulldogs,107.954038,28.516913,79.437125,803,2173,23,4.967972,1.312329,3.655643
391,Liam Jones,Carlton,126.700699,49.642002,77.058696,716,1814,19,6.984603,2.736604,4.247999
547,Sam Frost,Hawthorn,111.299396,38.919572,72.379826,897,2045,22,5.442513,1.903158,3.539356
99,Charlie Ballard,Gold Coast,102.027017,32.976551,69.050466,802,1947,21,5.240217,1.693711,3.546506


In [40]:
player_ratings_2021.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
194,Harris Andrews,Brisbane Lions,199.096075,71.459961,127.636113,1109,2224,23,8.952162,3.213128,5.739034
557,Sam Taylor,Greater Western Sydney,122.807288,24.724011,98.083278,829,1781,19,6.895412,1.388209,5.507203
540,Sam Collins,Gold Coast,112.503566,25.839008,86.664558,725,1788,20,6.292146,1.445135,4.847011
18,Aliir Aliir,Port Adelaide,181.933166,76.006224,105.926944,1042,2315,24,7.858884,3.283206,4.575678
391,Liam Jones,Carlton,126.700699,49.642002,77.058696,716,1814,19,6.984603,2.736604,4.247999
73,Brennan Cox,Fremantle,85.340186,39.938287,45.401899,459,1085,12,7.865455,3.680948,4.184507
263,Jake Lever,Melbourne,183.445366,82.954609,100.490758,1141,2472,25,7.420929,3.355769,4.065160
617,Tom Jonas,Port Adelaide,131.264200,43.761497,87.502701,985,2297,24,5.714593,1.905159,3.809434
419,Marcus Adams,Brisbane Lions,90.634364,31.631010,59.003355,682,1565,17,5.791333,2.021151,3.770182
13,Alex Keath,Western Bulldogs,107.954038,28.516913,79.437125,803,2173,23,4.967972,1.312329,3.655643


2022

In [41]:
player_ratings_2022.sort_values('vaep_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
582,Shai Bolton,Richmond,280.328637,293.403115,-13.074477,903,1985,23,14.122349,14.781013,-0.658664
110,Christian Petracca,Melbourne,261.054607,261.389177,-0.334570,1416,2108,24,12.383995,12.399866,-0.015871
383,Lachie Neale,Brisbane Lions,247.591694,244.988822,2.602874,1621,2182,25,11.347007,11.227719,0.119288
432,Marcus Bontempelli,Western Bulldogs,233.505681,234.801836,-1.296155,1158,1863,22,12.533853,12.603427,-0.069574
104,Charlie Curnow,Carlton,226.360849,242.881831,-16.520982,574,2022,22,11.194899,12.011960,-0.817061
97,Chad Warner,Sydney,221.593937,229.466990,-7.873055,1208,1938,24,11.434156,11.840402,-0.406246
211,Isaac Heeney,Sydney,217.044622,224.421783,-7.377162,896,2136,25,10.161265,10.506638,-0.345373
112,Clayton Oliver,Melbourne,215.824836,194.904025,20.920811,1629,1995,23,10.818288,9.769625,1.048662
626,Tom J. Lynch,Richmond,214.688405,236.221105,-21.532700,514,1601,19,13.409644,14.754597,-1.344953
500,Noah Anderson,Gold Coast,203.887293,192.143631,11.743662,1199,1837,21,11.098927,10.459642,0.639285


In [42]:
player_ratings_2022.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
582,Shai Bolton,Richmond,280.328637,293.403115,-13.074477,903,1985,23,14.122349,14.781013,-0.658664
626,Tom J. Lynch,Richmond,214.688405,236.221105,-21.532700,514,1601,19,13.409644,14.754597,-1.344953
432,Marcus Bontempelli,Western Bulldogs,233.505681,234.801836,-1.296155,1158,1863,22,12.533853,12.603427,-0.069574
110,Christian Petracca,Melbourne,261.054607,261.389177,-0.334570,1416,2108,24,12.383995,12.399866,-0.015871
184,George Hewett,Carlton,137.833854,124.478015,13.355837,913,1159,15,11.892481,10.740122,1.152359
514,Patrick Cripps,Carlton,202.032381,203.837196,-1.804814,1252,1707,21,11.835523,11.941253,-0.105730
450,Matthew Kennedy,Carlton,139.280242,129.653849,9.626393,897,1209,17,11.520285,10.724057,0.796228
97,Chad Warner,Sydney,221.593937,229.466990,-7.873055,1208,1938,24,11.434156,11.840402,-0.406246
383,Lachie Neale,Brisbane Lions,247.591694,244.988822,2.602874,1621,2182,25,11.347007,11.227719,0.119288
628,Tom Liberatore,Western Bulldogs,198.845076,172.274075,26.571000,1170,1753,22,11.343130,9.827386,1.515744


In [43]:
player_ratings_2022.sort_values('offensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
582,Shai Bolton,Richmond,280.328637,293.403115,-13.074477,903,1985,23,14.122349,14.781013,-0.658664
110,Christian Petracca,Melbourne,261.054607,261.389177,-0.334570,1416,2108,24,12.383995,12.399866,-0.015871
383,Lachie Neale,Brisbane Lions,247.591694,244.988822,2.602874,1621,2182,25,11.347007,11.227719,0.119288
104,Charlie Curnow,Carlton,226.360849,242.881831,-16.520982,574,2022,22,11.194899,12.011960,-0.817061
626,Tom J. Lynch,Richmond,214.688405,236.221105,-21.532700,514,1601,19,13.409644,14.754597,-1.344953
432,Marcus Bontempelli,Western Bulldogs,233.505681,234.801836,-1.296155,1158,1863,22,12.533853,12.603427,-0.069574
97,Chad Warner,Sydney,221.593937,229.466990,-7.873055,1208,1938,24,11.434156,11.840402,-0.406246
211,Isaac Heeney,Sydney,217.044622,224.421783,-7.377162,896,2136,25,10.161265,10.506638,-0.345373
623,Tom Hawkins,Geelong,191.685357,221.144217,-29.458860,696,2366,25,8.101663,9.346755,-1.245091
3,Aaron Naughton,Western Bulldogs,198.785415,216.741801,-17.956387,545,1899,22,10.467900,11.413470,-0.945571


In [44]:
player_ratings_2022.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
582,Shai Bolton,Richmond,280.328637,293.403115,-13.074477,903,1985,23,14.122349,14.781013,-0.658664
626,Tom J. Lynch,Richmond,214.688405,236.221105,-21.532700,514,1601,19,13.409644,14.754597,-1.344953
432,Marcus Bontempelli,Western Bulldogs,233.505681,234.801836,-1.296155,1158,1863,22,12.533853,12.603427,-0.069574
110,Christian Petracca,Melbourne,261.054607,261.389177,-0.334570,1416,2108,24,12.383995,12.399866,-0.015871
104,Charlie Curnow,Carlton,226.360849,242.881831,-16.520982,574,2022,22,11.194899,12.011960,-0.817061
514,Patrick Cripps,Carlton,202.032381,203.837196,-1.804814,1252,1707,21,11.835523,11.941253,-0.105730
97,Chad Warner,Sydney,221.593937,229.466990,-7.873055,1208,1938,24,11.434156,11.840402,-0.406246
427,Mabior Chol,Gold Coast,194.230156,204.265357,-10.035200,433,1777,22,10.930228,11.494955,-0.564727
3,Aaron Naughton,Western Bulldogs,198.785415,216.741801,-17.956387,545,1899,22,10.467900,11.413470,-0.945571
515,Patrick Dangerfield,Geelong,152.325448,150.919270,1.406178,862,1344,19,11.333739,11.229112,0.104626


In [45]:
player_ratings_2022.sort_values('defensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
573,Sam Taylor,Greater Western Sydney,157.556803,45.229689,112.327113,1018,2132,22,7.390094,2.121468,5.268626
144,Darcy Moore,Collingwood,187.126200,78.534718,108.591482,1044,2314,24,8.086698,3.393894,4.692804
555,Sam Collins,Gold Coast,128.009294,39.757235,88.252059,798,2154,22,5.942864,1.845740,4.097124
313,Jeremy Howe,Collingwood,142.085364,59.135689,82.949677,1051,2214,24,6.417586,2.670989,3.746598
615,Tom Barrass,West Coast,111.561756,28.810421,82.751335,865,1767,19,6.313625,1.630471,4.683154
629,Tom McCartin,Sydney,110.375907,30.010917,80.364989,825,2351,25,4.694849,1.276517,3.418332
188,Harris Andrews,Brisbane Lions,104.178809,29.246178,74.932631,948,2337,24,4.457801,1.251441,3.206360
558,Sam De Koning,Geelong,119.283010,48.756197,70.526813,825,2114,23,5.642526,2.306348,3.336178
19,Aliir Aliir,Port Adelaide,93.216651,23.805508,69.411144,756,1797,19,5.187348,1.324736,3.862612
592,Steven May,Melbourne,131.320822,63.427169,67.893653,1106,2049,22,6.409020,3.095518,3.313502


In [46]:
player_ratings_2022.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
573,Sam Taylor,Greater Western Sydney,157.556803,45.229689,112.327113,1018,2132,22,7.390094,2.121468,5.268626
144,Darcy Moore,Collingwood,187.126200,78.534718,108.591482,1044,2314,24,8.086698,3.393894,4.692804
615,Tom Barrass,West Coast,111.561756,28.810421,82.751335,865,1767,19,6.313625,1.630471,4.683154
162,Dylan Grimes,Richmond,73.924439,17.910024,56.014415,481,1228,15,6.019905,1.458471,4.561434
555,Sam Collins,Gold Coast,128.009294,39.757235,88.252059,798,2154,22,5.942864,1.845740,4.097124
617,Tom Clurey,Port Adelaide,62.645323,14.252772,48.392551,392,1182,13,5.299943,1.205818,4.094124
255,Jacob Weitering,Carlton,85.140155,18.696852,66.443303,644,1680,18,5.067866,1.112908,3.954959
19,Aliir Aliir,Port Adelaide,93.216651,23.805508,69.411144,756,1797,19,5.187348,1.324736,3.862612
313,Jeremy Howe,Collingwood,142.085364,59.135689,82.949677,1051,2214,24,6.417586,2.670989,3.746598
48,Ben McKay,North Melbourne,67.895079,24.999273,42.895806,428,1186,15,5.724712,2.107865,3.616847


2023

In [65]:
player_ratings_2023.sort_values('vaep_value', ascending = False).head(20)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
401,Marcus Bontempelli,Western Bulldogs,215.357107,203.393553,11.963554,1086,1519,18,14.177558,13.389964,0.787594
100,Charlie Curnow,Carlton,207.106687,222.511626,-15.404938,557,1677,18,12.349832,13.268433,-0.918601
543,Shai Bolton,Richmond,202.655066,199.882638,2.772428,835,1473,17,13.757981,13.569765,0.188216
105,Christian Petracca,Melbourne,192.157257,196.293392,-4.136135,1105,1508,18,12.742524,13.016803,-0.274280
465,Noah Anderson,Gold Coast,185.173663,177.060184,8.113479,1075,1525,18,12.142535,11.610504,0.532031
111,Connor Rozee,Port Adelaide,180.770880,182.782599,-2.011719,988,1509,18,11.979515,12.112830,-0.133315
4,Adam Cerra,Carlton,169.593779,151.636520,17.957259,1006,1304,17,13.005658,11.628567,1.377090
622,Zak Butters,Port Adelaide,166.915032,157.271575,9.643457,1056,1415,18,11.796115,11.114599,0.681516
504,Rory Laird,Adelaide,161.325950,141.545463,19.780487,1044,1340,17,12.039250,10.563094,1.476156
79,Caleb Serong,Fremantle,161.277859,152.403268,8.874591,1105,1420,17,11.357596,10.732625,0.624971


In [48]:
player_ratings_2023.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
401,Marcus Bontempelli,Western Bulldogs,215.357107,203.393553,11.963554,1086,1519,18,14.177558,13.389964,0.787594
543,Shai Bolton,Richmond,202.655066,199.882638,2.772428,835,1473,17,13.757981,13.569765,0.188216
4,Adam Cerra,Carlton,169.593779,151.636520,17.957259,1006,1304,17,13.005658,11.628567,1.377090
105,Christian Petracca,Melbourne,192.157257,196.293392,-4.136135,1105,1508,18,12.742524,13.016803,-0.274280
141,Darcy Parish,Essendon,127.347155,124.937990,2.409163,863,1023,13,12.448402,12.212902,0.235500
100,Charlie Curnow,Carlton,207.106687,222.511626,-15.404938,557,1677,18,12.349832,13.268433,-0.918601
465,Noah Anderson,Gold Coast,185.173663,177.060184,8.113479,1075,1525,18,12.142535,11.610504,0.532031
504,Rory Laird,Adelaide,161.325950,141.545463,19.780487,1044,1340,17,12.039250,10.563094,1.476156
111,Connor Rozee,Port Adelaide,180.770880,182.782599,-2.011719,988,1509,18,11.979515,12.112830,-0.133315
622,Zak Butters,Port Adelaide,166.915032,157.271575,9.643457,1056,1415,18,11.796115,11.114599,0.681516


In [49]:
player_ratings_2023.sort_values('offensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
100,Charlie Curnow,Carlton,207.106687,222.511626,-15.404938,557,1677,18,12.349832,13.268433,-0.918601
401,Marcus Bontempelli,Western Bulldogs,215.357107,203.393553,11.963554,1086,1519,18,14.177558,13.389964,0.787594
543,Shai Bolton,Richmond,202.655066,199.882638,2.772428,835,1473,17,13.757981,13.569765,0.188216
105,Christian Petracca,Melbourne,192.157257,196.293392,-4.136135,1105,1508,18,12.742524,13.016803,-0.274280
111,Connor Rozee,Port Adelaide,180.770880,182.782599,-2.011719,988,1509,18,11.979515,12.112830,-0.133315
465,Noah Anderson,Gold Coast,185.173663,177.060184,8.113479,1075,1525,18,12.142535,11.610504,0.532031
554,Taylor Walker,Adelaide,138.263277,157.729729,-19.466452,427,1383,17,9.997345,11.404897,-1.407553
622,Zak Butters,Port Adelaide,166.915032,157.271575,9.643457,1056,1415,18,11.796115,11.114599,0.681516
79,Caleb Serong,Fremantle,161.277859,152.403268,8.874591,1105,1420,17,11.357596,10.732625,0.624971
4,Adam Cerra,Carlton,169.593779,151.636520,17.957259,1006,1304,17,13.005658,11.628567,1.377090


In [50]:
player_ratings_2023.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
543,Shai Bolton,Richmond,202.655066,199.882638,2.772428,835,1473,17,13.757981,13.569765,0.188216
401,Marcus Bontempelli,Western Bulldogs,215.357107,203.393553,11.963554,1086,1519,18,14.177558,13.389964,0.787594
100,Charlie Curnow,Carlton,207.106687,222.511626,-15.404938,557,1677,18,12.349832,13.268433,-0.918601
105,Christian Petracca,Melbourne,192.157257,196.293392,-4.136135,1105,1508,18,12.742524,13.016803,-0.274280
141,Darcy Parish,Essendon,127.347155,124.937990,2.409163,863,1023,13,12.448402,12.212902,0.235500
111,Connor Rozee,Port Adelaide,180.770880,182.782599,-2.011719,988,1509,18,11.979515,12.112830,-0.133315
91,Cameron Zurhaar,North Melbourne,141.614758,144.305922,-2.691166,551,1238,15,11.438995,11.656375,-0.217380
4,Adam Cerra,Carlton,169.593779,151.636520,17.957259,1006,1304,17,13.005658,11.628567,1.377090
465,Noah Anderson,Gold Coast,185.173663,177.060184,8.113479,1075,1525,18,12.142535,11.610504,0.532031
554,Taylor Walker,Adelaide,138.263277,157.729729,-19.466452,427,1383,17,9.997345,11.404897,-1.407553


In [51]:
player_ratings_2023.sort_values('defensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
17,Aliir Aliir,Port Adelaide,132.287577,45.380588,86.906989,680,1759,18,7.520613,2.579908,4.940704
96,Charlie Ballard,Gold Coast,117.816296,39.661045,78.155252,714,1717,18,6.861753,2.309904,4.551849
462,Nick Murray,Adelaide,107.173299,38.346656,68.826643,520,1353,17,7.921160,2.834195,5.086966
517,Sam Collins,Gold Coast,100.488206,35.740131,64.748076,714,1755,18,5.725824,2.036475,3.689349
214,Jack Buckley,Greater Western Sydney,114.259959,49.921693,64.338265,701,1592,17,7.177133,3.135785,4.041348
14,Alex Pearce,Fremantle,93.732027,30.815106,62.916922,537,1694,18,5.533178,1.819074,3.714104
569,Tom Barrass,West Coast,81.958359,19.268944,62.689415,572,1341,14,6.111734,1.436909,4.674826
140,Darcy Moore,Collingwood,126.481592,64.899636,61.581956,852,1600,17,7.905100,4.056227,3.848872
151,Dougal Howard,St Kilda,86.593563,27.815268,58.778295,648,1595,17,5.429064,1.743904,3.685160
182,Harris Andrews,Brisbane Lions,103.514985,45.105010,58.409975,768,1656,17,6.250905,2.723732,3.527172


In [52]:
player_ratings_2023.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
462,Nick Murray,Adelaide,107.173299,38.346656,68.826643,520,1353,17,7.921160,2.834195,5.086966
17,Aliir Aliir,Port Adelaide,132.287577,45.380588,86.906989,680,1759,18,7.520613,2.579908,4.940704
569,Tom Barrass,West Coast,81.958359,19.268944,62.689415,572,1341,14,6.111734,1.436909,4.674826
96,Charlie Ballard,Gold Coast,117.816296,39.661045,78.155252,714,1717,18,6.861753,2.309904,4.551849
214,Jack Buckley,Greater Western Sydney,114.259959,49.921693,64.338265,701,1592,17,7.177133,3.135785,4.041348
140,Darcy Moore,Collingwood,126.481592,64.899636,61.581956,852,1600,17,7.905100,4.056227,3.848872
14,Alex Pearce,Fremantle,93.732027,30.815106,62.916922,537,1694,18,5.533178,1.819074,3.714104
169,Esava Ratugolea,Geelong,66.048653,23.977668,42.070986,392,1139,13,5.798828,2.105151,3.693677
517,Sam Collins,Gold Coast,100.488206,35.740131,64.748076,714,1755,18,5.725824,2.036475,3.689349
151,Dougal Howard,St Kilda,86.593563,27.815268,58.778295,648,1595,17,5.429064,1.743904,3.685160


In [53]:
action_vaep = get_vaep_action_summary(exp_vaep_2021)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-648.296156,-361.102346,-287.193813,972,0.002816,-0.666971,-0.371504,-0.295467,-0.014225,-0.008838,-0.060926
5,Handball,-3451.755728,-1733.225012,-1718.530714,60799,0.176132,-0.056773,-0.028507,-0.028266,-0.075741,-0.042419,-0.364575
12,Tackle,-2.146875,3.286310,-5.433185,129,0.000374,-0.016642,0.025475,-0.042118,-0.000047,0.000080,-0.001153
7,Kick,673.788592,3329.060663,-2655.272081,80643,0.233619,0.008355,0.041281,-0.032926,0.014785,0.081476,-0.563299
0,Carry,4279.316212,5135.636185,-856.319974,54634,0.158272,0.078327,0.094001,-0.015674,0.093899,0.125690,-0.181663
4,Gather,1788.390974,1566.473826,221.917147,14029,0.040641,0.127478,0.111660,0.015818,0.039242,0.038338,0.047078
11,Spoil,2082.077024,314.519537,1767.557494,13590,0.039370,0.153207,0.023143,0.130063,0.045686,0.007698,0.374976
13,Uncontested Mark,7088.531577,6704.717688,383.813886,38669,0.112022,0.183313,0.173387,0.009926,0.155541,0.164091,0.081424
6,Hard Ball Get,3825.577746,2402.135446,1423.442296,18947,0.054889,0.201909,0.126782,0.075128,0.083943,0.058790,0.301974
9,Loose Ball Get,9158.336102,6204.598507,2953.737601,34567,0.100139,0.264944,0.179495,0.085450,0.200958,0.151852,0.626616


In [54]:
action_vaep = get_vaep_action_summary(exp_vaep_2022)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-750.009181,-403.181244,-346.827937,1198,0.003503,-0.626051,-0.336545,-0.289506,-0.015985,-0.009599,-0.070562
12,Tackle,-8.876192,-6.015138,-2.861055,84,0.000246,-0.105669,-0.071609,-0.034060,-0.000189,-0.000143,-0.000582
5,Handball,-3556.634595,-1792.884149,-1763.750450,59270,0.173288,-0.060007,-0.030249,-0.029758,-0.075805,-0.042684,-0.358837
7,Kick,765.301723,3545.759470,-2780.457746,79965,0.233794,0.009570,0.044341,-0.034771,0.016311,0.084416,-0.565687
0,Carry,4454.790058,5209.150149,-754.360092,53272,0.155752,0.083623,0.097784,-0.014161,0.094947,0.124018,-0.153475
11,Spoil,2114.795675,300.007234,1814.788440,13964,0.040827,0.151446,0.021484,0.129962,0.045074,0.007142,0.369220
4,Gather,2040.652052,1730.134516,310.517534,13390,0.039148,0.152401,0.129211,0.023190,0.043494,0.041190,0.063175
13,Uncontested Mark,7496.846117,7074.774361,422.071762,38101,0.111396,0.196762,0.185685,0.011078,0.159784,0.168434,0.085871
6,Hard Ball Get,3409.097353,2146.631162,1262.466190,17178,0.050223,0.198457,0.124964,0.073493,0.072660,0.051106,0.256850
9,Loose Ball Get,9696.188123,6531.750635,3164.437489,37170,0.108674,0.260861,0.175726,0.085134,0.206660,0.155506,0.643808


In [55]:
action_vaep = get_vaep_action_summary(exp_vaep_2023)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-501.771636,-280.638659,-221.132977,805,0.003088,-0.623319,-0.348619,-0.274699,-0.013967,-0.008719,-0.059159
5,Handball,-3127.431586,-1751.335214,-1376.096365,46504,0.178405,-0.067251,-0.037660,-0.029591,-0.087052,-0.054409,-0.368142
12,Tackle,-1.616806,5.267518,-6.884324,80,0.000307,-0.020210,0.065844,-0.086054,-0.000045,0.000164,-0.001842
7,Kick,284.274276,2478.782570,-2194.508295,59830,0.229528,0.004751,0.041430,-0.036679,0.007913,0.077009,-0.587089
0,Carry,4072.605238,4573.713132,-501.107893,41649,0.159780,0.097784,0.109816,-0.012032,0.113361,0.142093,-0.134060
4,Gather,1525.463746,1289.688895,235.774848,10263,0.039372,0.148637,0.125664,0.022973,0.042461,0.040067,0.063076
11,Spoil,1695.463174,293.353672,1402.109504,10423,0.039986,0.162666,0.028145,0.134521,0.047193,0.009114,0.375101
6,Hard Ball Get,2594.279941,1614.638264,979.641675,12498,0.047947,0.207576,0.129192,0.078384,0.072212,0.050162,0.262080
13,Uncontested Mark,6023.574122,5661.682206,361.891913,28503,0.109347,0.211331,0.198635,0.012697,0.167666,0.175893,0.096816
9,Loose Ball Get,7974.440809,5363.606526,2610.834282,29260,0.112251,0.272537,0.183308,0.089229,0.221968,0.166633,0.698467


In [56]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-30.446724,-20.193816,-10.252908,283,0.198457,-0.107586,-0.071356,-0.036229,-0.289372,-0.196674,-4.036510
1,Error,-4.285456,-2.139392,-2.146064,6,0.004208,-0.714243,-0.356565,-0.357677,-0.040730,-0.020836,-0.844893
6,Kick,-2.470649,5.729843,-8.200492,344,0.241234,-0.007182,0.016657,-0.023839,-0.023482,0.055805,-3.228485
10,Spoil,-0.760046,0.415165,-1.175211,5,0.003506,-0.152009,0.083033,-0.235042,-0.007224,0.004043,-0.462674
7,Knock On,-0.467319,-0.969665,0.502346,4,0.002805,-0.116830,-0.242416,0.125586,-0.004442,-0.009444,0.197771
3,Gather,4.819729,2.947091,1.872639,44,0.030856,0.109539,0.066979,0.042560,0.045808,0.028703,0.737247
5,Hard Ball Get,11.231130,8.730428,2.500702,47,0.032959,0.238960,0.185754,0.053206,0.106743,0.085029,0.984512
11,Uncontested Mark,12.201581,11.104485,1.097096,124,0.086957,0.098400,0.089552,0.008848,0.115966,0.108150,0.431920
9,Shot,15.015949,15.083288,-0.067339,17,0.011921,0.883291,0.887252,-0.003961,0.142715,0.146901,-0.026511
2,Free For,25.472519,13.980282,11.492238,28,0.019635,0.909733,0.499296,0.410437,0.242096,0.136159,4.524427


In [57]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-10.638943,-3.564828,-7.074115,271,0.233018,-0.039258,-0.013154,-0.026104,-0.074501,-0.025470,-2.490474
1,Error,-0.809905,-0.704833,-0.105073,1,0.000860,-0.809905,-0.704833,-0.105073,-0.005672,-0.005036,-0.036991
7,Knock On,-0.581205,-0.796839,0.215634,2,0.001720,-0.290602,-0.398419,0.107817,-0.004070,-0.005693,0.075915
10,Spoil,0.765711,0.950649,-0.184938,3,0.002580,0.255237,0.316883,-0.061646,0.005362,0.006792,-0.065108
5,Hard Ball Get,3.653121,5.377233,-1.724112,30,0.025795,0.121771,0.179241,-0.057470,0.025582,0.038419,-0.606981
3,Gather,7.225288,5.907249,1.318038,49,0.042132,0.147455,0.120556,0.026899,0.050596,0.042206,0.464021
6,Kick,9.330977,13.586047,-4.255069,245,0.210662,0.038086,0.055453,-0.017368,0.065342,0.097070,-1.498016
11,Uncontested Mark,10.008875,9.871835,0.137041,59,0.050731,0.169642,0.167319,0.002323,0.070089,0.070532,0.048246
2,Free For,11.006062,9.022914,1.983148,22,0.018917,0.500276,0.410132,0.090143,0.077072,0.064467,0.698176
9,Shot,29.260832,29.816676,-0.555844,34,0.029235,0.860613,0.876961,-0.016348,0.204904,0.213034,-0.195687


In [58]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-10.909260,2.069228,-12.978488,316,0.223164,-0.034523,0.006548,-0.041071,-0.041789,0.007916,38.791594
2,Error,-4.020583,-2.193743,-1.826840,6,0.004237,-0.670097,-0.365624,-0.304473,-0.015401,-0.008393,5.460268
8,Knock On,1.064300,1.055248,0.009052,6,0.004237,0.177383,0.175875,0.001509,0.004077,0.004037,-0.027056
1,Contested Mark,1.334488,1.259660,0.074828,4,0.002825,0.333622,0.314915,0.018707,0.005112,0.004819,-0.223653
11,Spoil,1.440981,0.979395,0.461586,5,0.003531,0.288196,0.195879,0.092317,0.005520,0.003747,-1.379641
0,Carry,13.782604,17.447091,-3.664486,232,0.163842,0.059408,0.075203,-0.015795,0.052796,0.066748,10.952836
3,Free For,15.019876,14.414790,0.605087,16,0.011299,0.938742,0.900924,0.037818,0.057535,0.055147,-1.808553
6,Hard Ball Get,24.399315,19.922286,4.477029,102,0.072034,0.239209,0.195317,0.043892,0.093464,0.076217,-13.381457
12,Uncontested Mark,28.150172,27.944843,0.205329,103,0.072740,0.273303,0.271309,0.001993,0.107833,0.106909,-0.613710
4,Gather,34.306015,28.115829,6.190186,112,0.079096,0.306304,0.251034,0.055270,0.131413,0.107563,-18.501937


In [59]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-25.633785,-16.005286,-9.628500,242,0.219005,-0.105925,-0.066138,-0.039787,-0.133400,-0.081538,2.327898
2,Error,-6.368553,-3.276802,-3.091751,11,0.009955,-0.578959,-0.297891,-0.281068,-0.033142,-0.016693,0.747498
8,Knock On,-0.062214,-0.590883,0.528668,2,0.001810,-0.031107,-0.295441,0.264334,-0.000324,-0.003010,-0.127817
11,Spoil,1.826901,1.061243,0.765658,5,0.004525,0.365380,0.212249,0.153132,0.009507,0.005406,-0.185114
1,Contested Mark,5.906266,5.956578,-0.050312,4,0.003620,1.476567,1.489145,-0.012578,0.030737,0.030345,0.012164
6,Hard Ball Get,12.907052,10.820845,2.086206,85,0.076923,0.151848,0.127304,0.024544,0.067169,0.055126,-0.504385
3,Free For,16.219864,13.619663,2.600201,18,0.016290,0.901104,0.756648,0.144456,0.084409,0.069384,-0.628655
4,Gather,16.951466,14.792039,2.159427,76,0.068778,0.223046,0.194632,0.028414,0.088217,0.075357,-0.522088
0,Carry,17.639628,20.421156,-2.781528,167,0.151131,0.105627,0.122282,-0.016656,0.091798,0.104034,0.672494
12,Uncontested Mark,22.128719,21.730024,0.398694,83,0.075113,0.266611,0.261808,0.004804,0.115159,0.110702,-0.096393


In [60]:
team_data = get_team(exp_vaep_2022, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-278.707185,-159.679733,-119.027453,3861,0.181310,-0.072185,-0.041357,-0.030828,-0.092081,-0.058214,-0.419437
2,Error,-58.554754,-36.543664,-22.011090,76,0.003569,-0.770457,-0.480838,-0.289620,-0.019346,-0.013323,-0.077564
12,Tackle,-0.783123,-0.279061,-0.504062,3,0.000141,-0.261041,-0.093020,-0.168021,-0.000259,-0.000102,-0.001776
8,Knock On,76.751993,55.349072,21.402922,221,0.010378,0.347294,0.250448,0.096846,0.025358,0.020178,0.075421
11,Spoil,103.963404,14.222623,89.740782,762,0.035783,0.136435,0.018665,0.117770,0.034348,0.005185,0.316235
4,Gather,109.322426,93.077810,16.244615,861,0.040432,0.126971,0.108104,0.018867,0.036119,0.033933,0.057244
6,Hard Ball Get,206.477620,127.438132,79.039487,1042,0.048932,0.198155,0.122301,0.075854,0.068217,0.046460,0.278525
7,Kick,211.986002,320.628829,-108.642828,4833,0.226955,0.043862,0.066342,-0.022479,0.070037,0.116890,-0.382843
1,Contested Mark,251.325352,190.956105,60.369246,387,0.018173,0.649420,0.493427,0.155993,0.083034,0.069616,0.212733
0,Carry,315.573445,368.893539,-53.320094,3308,0.155342,0.095397,0.111516,-0.016119,0.104261,0.134486,-0.187893


In [61]:
team_data = get_team(exp_vaep_2023, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-219.180523,-122.403406,-96.777117,2425,0.171197,-0.090384,-0.050476,-0.039908,-0.105913,-0.064104,-0.604929
2,Error,-32.335771,-21.638250,-10.697521,45,0.003177,-0.718573,-0.480850,-0.237723,-0.015625,-0.011332,-0.066868
12,Tackle,-1.250243,0.559766,-1.810009,4,0.000282,-0.312561,0.139941,-0.452502,-0.000604,0.000293,-0.011314
8,Knock On,37.642614,28.347093,9.295521,106,0.007483,0.355119,0.267425,0.087694,0.018190,0.014846,0.058104
4,Gather,78.769107,71.341173,7.427934,520,0.036710,0.151479,0.137195,0.014284,0.038063,0.037362,0.046430
7,Kick,86.968443,193.808058,-106.839612,3322,0.234522,0.026180,0.058341,-0.032161,0.042025,0.101499,-0.667828
11,Spoil,111.534634,27.264113,84.270521,532,0.037557,0.209652,0.051248,0.158403,0.053896,0.014279,0.526754
6,Hard Ball Get,134.235223,95.182173,39.053049,682,0.048147,0.196826,0.139563,0.057263,0.064866,0.049848,0.244111
0,Carry,188.819779,217.954220,-29.134441,2112,0.149100,0.089403,0.103198,-0.013795,0.091242,0.114145,-0.182112
1,Contested Mark,196.423494,164.127852,32.295644,294,0.020755,0.668107,0.558258,0.109849,0.094917,0.085956,0.201872


In [62]:
team_data = get_team(exp_vaep_2023, "Collingwood")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-217.331125,-137.988693,-79.342432,2561,0.184112,-0.084862,-0.053881,-0.030981,-0.110150,-0.075969,-0.506446
2,Error,-31.491659,-16.815431,-14.676228,51,0.003666,-0.617484,-0.329714,-0.287769,-0.015961,-0.009258,-0.093679
12,Tackle,-0.963187,-0.238470,-0.724717,3,0.000216,-0.321062,-0.079490,-0.241572,-0.000488,-0.000131,-0.004626
7,Kick,35.701835,154.543173,-118.841338,3129,0.224946,0.011410,0.049391,-0.037981,0.018095,0.085083,-0.758570
8,Knock On,55.955107,40.094215,15.860891,201,0.014450,0.278384,0.199474,0.078910,0.028360,0.022074,0.101241
11,Spoil,76.557590,15.664350,60.893239,506,0.036377,0.151300,0.030957,0.120342,0.038802,0.008624,0.388684
4,Gather,78.685079,69.381543,9.303535,572,0.041121,0.137561,0.121296,0.016265,0.039880,0.038198,0.059385
6,Hard Ball Get,113.786781,72.364958,41.421822,574,0.041265,0.198235,0.126071,0.072163,0.057671,0.039840,0.264397
1,Contested Mark,197.894942,149.625784,48.269158,271,0.019482,0.730240,0.552125,0.178115,0.100300,0.082376,0.308104
10,Shot,282.796781,300.586276,-17.789497,432,0.031057,0.654622,0.695802,-0.041179,0.143331,0.165487,-0.113551


In [63]:
team_data = get_team(exp_vaep_2023, "St Kilda")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-192.623115,-111.540777,-81.082338,2555,0.180975,-0.075391,-0.043656,-0.031735,-0.104617,-0.068864,-0.366072
7,Kick,-46.054232,81.216197,-127.270430,3328,0.235727,-0.013838,0.024404,-0.038242,-0.025013,0.050142,-0.574603
2,Error,-27.364909,-16.761485,-10.603424,54,0.003825,-0.506758,-0.310398,-0.196360,-0.014862,-0.010348,-0.047873
12,Tackle,4.435548,1.821228,2.614320,4,0.000283,1.108887,0.455307,0.653580,0.002409,0.001124,0.011803
8,Knock On,43.670704,30.905602,12.765102,150,0.010625,0.291138,0.206037,0.085101,0.023718,0.019081,0.057632
4,Gather,57.866133,50.794225,7.071908,534,0.037824,0.108364,0.095120,0.013243,0.031428,0.031360,0.031928
11,Spoil,71.486703,16.805503,54.681200,499,0.035345,0.143260,0.033678,0.109582,0.038826,0.010375,0.246876
6,Hard Ball Get,144.403024,71.539057,72.863966,677,0.047953,0.213298,0.105671,0.107628,0.078428,0.044167,0.328968
1,Contested Mark,156.395149,115.630567,40.764583,212,0.015016,0.737713,0.545427,0.192286,0.084941,0.071389,0.184045
0,Carry,239.389857,242.346783,-2.956925,2298,0.162771,0.104173,0.105460,-0.001287,0.130016,0.149621,-0.013350


Match Expectd Scores v Expected VAEP

In [67]:
match_id = "2022F4_Geelong_Sydney"
match_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id)
match_vaep_ranking.sort_values('vaep_value', ascending = False).head()

,player,team,vaep_value,offensive_value,defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_vaep_value,normalised_offensive_value,normalised_defensive_value,z_vaep_value,z_offensive_value,z_defensive_value
5,Chad Warner,Sydney,13.569964,13.891235,-0.321271,69,1.0,1.0,35.0,1.000000,1.000000,0.302138,2.448048,2.555069,-0.506856
24,Mark Blicavs,Geelong,12.392599,11.351990,1.040609,49,2.0,4.0,10.0,0.920200,0.830159,0.492781,2.127088,1.889899,0.385276
13,Isaac Smith,Geelong,12.125486,13.375887,-1.250401,69,3.0,2.0,42.0,0.902095,0.965530,0.172073,2.054271,2.420071,-1.115504
30,Patrick Dangerfield,Geelong,11.997718,12.976029,-0.978310,58,4.0,3.0,40.0,0.893435,0.938785,0.210162,2.019440,2.315326,-0.937264
4,Cameron Guthrie,Geelong,10.065827,9.639613,0.426214,36,5.0,6.0,22.0,0.762494,0.715624,0.406775,1.492790,1.441332,-0.017198


In [76]:
match_chain = get_match(exp_vaep_data, match_id)
match_chain.head(15)

,match_id,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,scores,concedes,offensive_value,defensive_value,vaep_value,Season
685607,2022F4_Geelong_Sydney,1,5.0,5.0,Geelong,Patrick Dangerfield,1.0,-2.0,1.0,-2.0,Gather,effective,0.000000,0.184905,0.080090,0.000000,-0.000000,0.000000,2022
685608,2022F4_Geelong_Sydney,1,5.0,5.0,Geelong,Patrick Dangerfield,1.0,-2.0,17.0,-3.0,Kick,ineffective,0.000000,1.138233,0.048147,0.953328,0.031943,0.985271,2022
685609,2022F4_Geelong_Sydney,1,8.0,8.0,Geelong,Rhys Stanley,17.0,-3.0,18.0,-2.0,Loose Ball Get,effective,0.000000,0.839802,0.029436,-0.298431,0.018711,-0.279720,2022
685610,2022F4_Geelong_Sydney,1,9.0,9.0,Geelong,Rhys Stanley,18.0,-2.0,-21.0,-3.0,Handball,clanger,0.000000,0.376948,0.462636,-0.462854,-0.433200,-0.896054,2022
685611,2022F4_Geelong_Sydney,1,13.0,13.0,Sydney,Luke Parker,-21.0,-3.0,-25.0,-6.0,Hard Ball Get,effective,0.000000,0.515518,0.116540,0.052882,0.260408,0.313290,2022
685612,2022F4_Geelong_Sydney,1,13.0,13.0,Sydney,Luke Parker,-25.0,-6.0,-32.0,-11.0,Handball,effective,0.000000,0.925819,0.119335,0.410301,-0.002795,0.407507,2022
685613,2022F4_Geelong_Sydney,1,16.0,16.0,Sydney,Tom Papley,-32.0,-11.0,-37.0,-10.0,Carry,effective,0.000000,2.114803,0.114272,1.188984,0.005063,1.194047,2022
685614,2022F4_Geelong_Sydney,1,19.0,19.0,Sydney,Tom Papley,-37.0,-10.0,23.0,52.0,Kick,ineffective,0.000000,0.042203,0.225453,-2.072601,-0.111181,-2.183781,2022
685615,2022F4_Geelong_Sydney,1,24.0,24.0,Geelong,Zach Tuohy,23.0,52.0,-23.0,-52.0,Loose Ball Get,effective,0.000000,0.760515,0.011948,0.535062,0.030254,0.565317,2022
685616,2022F4_Geelong_Sydney,1,24.0,24.0,Geelong,Zach Tuohy,-23.0,-52.0,-30.0,-57.0,Loose Ball Get,effective,0.000000,1.008886,0.068501,0.248371,-0.056552,0.191818,2022


In [81]:
match_chain.groupby('team').sum()[['xScore', 'vaep_value', 'offensive_value', 'defensive_value']]

,xScore,vaep_value,offensive_value,defensive_value
team,,,,
Geelong,107.527074,135.954603,121.602793,14.351811
Sydney,43.391856,75.180133,68.718450,6.461684


In [79]:
match_shots = match_chain[match_chain['xScore'] > 0]
match_shots.groupby('team').sum()[['xScore','scores', 'concedes', 'vaep_value', 'offensive_value', 'defensive_value']]

,xScore,scores,concedes,vaep_value,offensive_value,defensive_value
team,,,,,,
Geelong,107.527074,112.463860,6.290064,26.273499,27.339403,-1.065904
Sydney,43.391856,45.690933,2.727993,11.321169,12.358562,-1.037393


In [80]:
shots = exp_vaep_data[exp_vaep_data['xScore'] > 0]
shots.sum()[['xScore','scores', 'concedes', 'vaep_value', 'offensive_value', 'defensive_value']]

xScore             94899.841369
scores             96652.552941
concedes            5017.917568
vaep_value         21503.367374
offensive_value    22121.611703
defensive_value     -618.244329
dtype: object